In [1]:
#Import Python packages
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import json
import datetime

# Import Snowflake modules
from snowflake.snowpark import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark import Window

In [2]:
# Get account credentials from a json file
with open("data_scientist_auth.json") as f:
    data = json.load(f)
    username = data["username"]
    password = data["password"]
    account = data["account"]

# Specify connection parameters
connection_parameters = {
    "account": account,
    "user": username,
    "password": password,
    "role": "TASTY_BI",
    "warehouse": "TASTY_BI_WH",
    "database": "frostbyte_tasty_bytes",
    "schema": "analytics",
}

# Create Snowpark session
session = Session.builder.configs(connection_parameters).create()

In [3]:
#Function to remove and rename columns after inner join
def remove_dup_join_col(df):
    # capitalise all columns
    for col in df.columns:
        df = df.withColumnRenamed(col, col.upper())
    
    # get list of renamable columns for left and right join
    left_dup_col = [col_name for col_name in df.columns if col_name.startswith('L_')]
    right_dup_col = [col_name for col_name in df.columns if col_name.startswith('R_')]
    old_columns = df.columns
    
    # rename the list with the most number of renamable columns
    # drop the list of columns with less number of renamable columns
    if len(left_dup_col)>len(right_dup_col):
        columns_rename=left_dup_col
        df= df.drop(*right_dup_col)
    else:
        columns_rename=right_dup_col
        df= df.drop(*left_dup_col)
    
    for old_column in old_columns:
        if old_column in columns_rename:
            # get string to remove
            string_to_replace = columns_rename[0][:7]
            # replace starting string
            new_column = old_column.replace(string_to_replace, "")
            df = df.withColumnRenamed(old_column, new_column)
    
    return df

Objective of this notebook is to rebuild a NPS_Compare or similar to do high level correlation

In [4]:
ORDER_HEADERS = session.table("FROSTBYTE_TASTY_BYTES.RAW_POS.ORDER_HEADER")
CUSTOMERS = session.table("FROSTBYTE_TASTY_BYTES.RAW_CUSTOMER.CUSTOMER_LOYALTY")

In [5]:
ORDER_HEADERS.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"CUSTOMER_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|369186517   |339         |11823.0        |NULL           |NULL           |200372773   |15:30:00            |22:30:00          |NULL             |2021-05-

In [6]:
CUSTOMERS.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CUSTOMER_ID"  |"FIRST_NAME"  |"LAST_NAME"  |"CITY"          |"COUNTRY"  |"POSTAL_CODE"  |"PREFERRED_LANGUAGE"  |"GENDER"     |"FAVOURITE_BRAND"  |"MARITAL_STATUS"  |"CHILDREN_COUNT"  |"SIGN_UP_DATE"  |"BIRTHDAY_DATE"  |"E_MAIL"                  |"PHONE_NUMBER"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1              |Rodolfo       |Tucker       |Hamburg         |Germany    |21129          |German                |Female       |NULL               |Single            |0                 |2019-02-19      

In [7]:
LOOKER = ORDER_HEADERS.join(CUSTOMERS, CUSTOMERS['CUSTOMER_ID'] == ORDER_HEADERS['CUSTOMER_ID'], 'inner')

LOOKER = remove_dup_join_col(LOOKER)

LOOKER.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |"CUSTOMER_ID"  |"FIRST_NAME"  |"LAST_NAME"  |"CITY"         |"COUNTRY"      |"POSTAL_CODE"  |"PREFERRED_LANGUAGE"  |"GENDER"     |"FAVOURITE_BRAND"  |"MARITAL_STATUS"    |"CHILDREN_COUNT"  |"SIGN_

In [8]:
# Define a window partitioned by customer and menu item, ordered by date
window_spec = Window.partitionBy("CUSTOMER_ID").orderBy("ORDER_TS")

# Add a column with the date of the prior order for each customer and order
orders_df_pre = LOOKER.withColumn("DTNO", F.lead("ORDER_TS").over(window_spec))

orders_df = orders_df_pre.withColumn("DTNO", F.datediff(col1="ORDER_TS", col2="DTNO",part='day'))
orders_df = orders_df.na.drop(subset=["DTNO"])

orders_df.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |"CUSTOMER_ID"  |"FIRST_NAME"  |"LAST_NAME"  |"CITY"  |"COUNTRY"  |"POSTAL_CODE"  |"PREFERRED_LANGUAGE"  |"GENDER"  |"FAVOURITE_BRAND"  |"MARITAL_STATUS"  |"CHILDREN_COUNT"  |"SIGN_UP_DATE"  |"BIRTHDAY_DA

In [9]:
orders_df.write.save_as_table(table_name="tran_anal", mode='overwrite')

In [5]:
orders_df = session.table("tran_anal")

In [6]:
orders_df.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |"CUSTOMER_ID"  |"FIRST_NAME"  |"LAST_NAME"  |"CITY"   |"COUNTRY"  |"POSTAL_CODE"  |"PREFERRED_LANGUAGE"  |"GENDER"  |"FAVOURITE_BRAND"  |"MARITAL_STATUS"  |"CHILDREN_COUNT"  |"SIGN_UP_DATE"  |"BIRTHDAY_DA

In [11]:
orders_df = orders_df.with_column("YEAR", F.year("ORDER_TS"))
orders_df = orders_df.with_column("MONTH", F.month("ORDER_TS"))
orders_df = orders_df.with_column("YEAR_MONTH", F.concat(F.col("YEAR"), F.col("MONTH")))
orders_df.show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |"CUSTOMER_ID"  |"FIRST_NAME"  |"LAST_NAME"  |"CITY"   |"COUNTRY"  |"POSTAL_CODE"  |"PREFERRED_LANGUAGE"  |"GENDER"  |"FAVOURITE_BRAND"  |"MARITAL_STATUS"  |"CHILDREN_COUN

In [12]:
AVG_SALES_YM = orders_df.groupBy(F.col("YEAR"),F.col("MONTH")).agg(F.avg('"ORDER_TOTAL"'))
SUM_SALES_YM = orders_df.groupBy(F.col("YEAR"),F.col("MONTH")).agg(F.sum('"ORDER_TOTAL"'))

AVG_SALES_YM.show()
SUM_SALES_YM.show()

-----------------------------------------
|"YEAR"  |"MONTH"  |"AVG(ORDER_TOTAL)"  |
-----------------------------------------
|2020    |6        |40.9888317457       |
|2020    |11       |41.9294751366       |
|2020    |3        |40.9417636844       |
|2020    |7        |41.5224986601       |
|2021    |9        |41.2782516812       |
|2019    |5        |34.9114314222       |
|2021    |7        |41.4256084795       |
|2020    |8        |41.4468220109       |
|2021    |12       |41.1608334972       |
|2020    |5        |40.8018857880       |
-----------------------------------------

-----------------------------------------
|"YEAR"  |"MONTH"  |"SUM(ORDER_TOTAL)"  |
-----------------------------------------
|2020    |6        |9440465.7500        |
|2021    |12       |19045652.7500       |
|2020    |12       |13399496.0000       |
|2021    |11       |18665928.2500       |
|2019    |4        |712290.0000         |
|2022    |7        |21768942.2500       |
|2020    |11       |12565341.2500

In [13]:
churn_YN = orders_df.with_column("CHURN", F.iff(F.col("DTNO") > 9, "1", "0"))

churn_YN.show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"           |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |"CUSTOMER_ID"  |"FIRST_NAME"  |"LAST_NAME"  |"CITY"  |"COUNTRY"  |"POSTAL_CODE"  |"PREFERRED_LANGUAGE"  |"GENDER"  |"FAVOURITE_BRAND"  |"MARITAL_STATUS"  |"CHILDR

In [20]:
churn_rate_YM = churn_YN.group_by(F.col("YEAR"), F.col("MONTH")).agg(F.sum("CHURN")/F.count("CUSTOMER_ID"))

churn_to_sales = churn_rate_YM.join(SUM_SALES_YM, (churn_rate_YM["YEAR"] == SUM_SALES_YM["YEAR"]) & (churn_rate_YM["MONTH"] == SUM_SALES_YM["MONTH"]), how="inner")

churn_to_sales = remove_dup_join_col(churn_to_sales)
churn_to_sales.show()

------------------------------------------------------------------------------------
|"DIVIDE(SUM(CHURN), COUNT(CUSTOMER_ID))"  |"YEAR"  |"MONTH"  |"SUM(ORDER_TOTAL)"  |
------------------------------------------------------------------------------------
|0.643117106215968                         |2020    |12       |13399496.0000       |
|0.5310078032590082                        |2021    |11       |18665928.2500       |
|0.9200656465704494                        |2019    |4        |712290.0000         |
|0.6468843225061566                        |2020    |11       |12565341.2500       |
|0.5354679898770944                        |2021    |12       |19045652.7500       |
|0.719340519323051                         |2020    |3        |7968823.0000        |
|0.5935371076936158                        |2021    |5        |15527795.5000       |
|0.5192629466278891                        |2022    |1        |19436288.5000       |
|0.5076328137705879                        |2022    |3        |20

In [34]:
churn_to_sales_push = churn_to_sales.select("YEAR", "MONTH", "DIVIDE(SUM(CHURN), COUNT(CUSTOMER_ID))", "SUM(ORDER_TOTAL)")

churn_to_sales_push = churn_to_sales_push.withColumnRenamed("DIVIDE(SUM(CHURN), COUNT(CUSTOMER_ID))","CHURN_RATE")
churn_to_sales_push = churn_to_sales_push.withColumnRenamed("SUM(ORDER_TOTAL)", "SALES")

churn_to_sales_push = churn_to_sales_push.orderBy("YEAR", "MONTH")

churn_to_sales_push.show()

--------------------------------------------------------
|"YEAR"  |"MONTH"  |"CHURN_RATE"        |"SALES"       |
--------------------------------------------------------
|2019    |1        |0.9816951850378034  |91308.5000    |
|2019    |2        |0.9666611101850309  |220793.5000   |
|2019    |3        |0.9417467398764585  |425460.0000   |
|2019    |4        |0.9200656465704494  |712290.0000   |
|2019    |5        |0.9008302747389801  |979719.5000   |
|2019    |6        |0.8789374410562716  |1109682.0000  |
|2019    |7        |0.8436551305403764  |1690853.5000  |
|2019    |8        |0.8197562776957164  |2024942.0000  |
|2019    |9        |0.7965229761471532  |2217703.0000  |
|2019    |10       |0.7886291932264503  |2920658.0000  |
--------------------------------------------------------



In [33]:
churn_to_sales_push.write.save_as_table(table_name="churn_to_sales", mode='overwrite')